In [35]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [51]:
# determine the supported device
def get_device():
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu') # don't have GPU 
    return device

# convert a df to tensor to be used in pytorch
def df_to_tensor(df):
    device = get_device()
    return torch.from_numpy(df.values).float().to(device)

In [52]:
get_device()

device(type='cpu')

In [53]:
X = pd.read_csv("bitcoin_train.csv",usecols=[7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 24, 27, 28, 29, 32, 33, 34])
y = pd.read_csv("bitcoin_train.csv",usecols=[35])

def y_2d(x):
    if x == 1.0:
        return 0
    return 1
    
#y['second_label'] = y.apply(lambda x: y_2d(x[0]), axis = 1)

X = X.to_numpy()
y = y.to_numpy()

X_train, X_valid, y_train, y_valid = train_test_split(X, y, random_state=1234)

In [73]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight.data)
        nn.init.normal_(m.bias.data)

class Net(nn.Module):

    def __init__(self, sizes):
        super(Net, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1]))
            layers.append(nn.Sigmoid())
        self.model = nn.Sequential(*layers)
        self.model.apply(init_weights)
        self.softmax = nn.Softmax(dim=0)

    def forward(self, x):
        return self.model(x)
        #pred = self.softmax(self.model.forward(x)).argmax()
        #if pred == 0:
        #    return torch.Tensor([1])
        #return torch.Tensor([0])
        
        

    def train(self, X_train, y_train, X_valid=None, y_valid=None,
          eta=0.25, num_epochs=10, isPrint=True):
        """
        Train the network with SGD 

        :param X_train: matrix of training features 
        :param y_train: matrix of vector-encoded labels 
        """
        criterion = nn.MSELoss()
        optimizer = optim.SGD(self.parameters(), lr=eta)
        # initialize shuffled indices 
        shuffled_inds = list(range(X_train.shape[0]))
        # loop over training epochs 
        for ep in range(1, num_epochs+1):

            # shuffle indices 
            np.random.shuffle(shuffled_inds)

            # loop over training examples 
            for ind in shuffled_inds:
                optimizer.zero_grad()
                output = self.forward(torch.from_numpy(X_train[ind]).float())
                loss = criterion(output, torch.from_numpy(y_train[ind]).float())
                loss.backward() 
                optimizer.step() 

            # print mean loss every 10 epochs if requested 
            if isPrint and (ep % 10) == 0:
                print("epoch {:3d}/{:3d}: ".format(ep, num_epochs), end="")
                print("  train loss: {:8.3f}".format(self.compute_loss(X_train, y_train)), end="")
                if X_valid is not None:
                    print("  validation loss: {:8.3f}".format(self.compute_loss(X_valid, y_valid)))
                else:
                    print("")

    def compute_loss(self, X, y):
        """
        compute average loss for given data set 
        
        :param X: matrix of features 
        :param y: matrix of vector-encoded labels 
        """
        loss = 0
        if len(X.shape) == 1:
            X = X[np.newaxis, :]
        if len(y.shape) == 1:
            y = y[np.newaxis, :]
        criterion = nn.MSELoss()
        for x, t in zip(X, y):
            output = self.forward(torch.from_numpy(x).float())
            loss += criterion(output, torch.from_numpy(t).float()).item()
        return loss / X.shape[0]
    

In [74]:
model = Net([X_train.shape[1], 3, 1])

In [75]:
model.train(X_train, y_train, X_valid, y_valid, eta=0.10, num_epochs = 100)

epoch  10/100:   train loss:    0.247  validation loss:    0.241
epoch  20/100:   train loss:    0.245  validation loss:    0.248
epoch  30/100:   train loss:    0.247  validation loss:    0.244
epoch  40/100:   train loss:    0.251  validation loss:    0.245
epoch  50/100:   train loss:    0.248  validation loss:    0.244
epoch  60/100:   train loss:    0.248  validation loss:    0.245
epoch  70/100:   train loss:    0.248  validation loss:    0.244
epoch  80/100:   train loss:    0.248  validation loss:    0.244
epoch  90/100:   train loss:    0.247  validation loss:    0.245
epoch 100/100:   train loss:    0.247  validation loss:    0.245


In [69]:
pred = model.forward(torch.from_numpy(X_valid[13]).float())
loss_1 = loss(torch.round(pred), torch.Tensor([1]))
loss_1.backward()

In [66]:
loss = nn.MSELoss()